In [ ]:
from google.colab import drive
drive.mount('/drive')
import pandas as pd
import numpy as np
import json
import ast
from tqdm.notebook import tqdm
tqdm.pandas()

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


## Prerequisite
1. Your HOME directory has following files:
- user_schedule.json
- courses.csv
<br/>
Prior to processing the user_schedule.json, you should already have the csv files. If you don't, please acquire those ones first.
<br/>
2. user_schedule.json has 6 columns: 
```
_id.$oid
user_id.$oid
name
deleted
schedule_items
created_at.$date
```
## Execution
Just `Run All` cells and you will get the csv result placed in your HOME directory


In [ ]:
HOME = "/drive/My Drive/Susun Jadwal/"

In [ ]:
f = open(HOME+"user_schedule.json", "r")
schedule = pd.json_normalize(json.load(f)).rename(columns={"_id.$oid":"id_schedule", "user_id.$oid": "user_id", "created_at.$date": "created_at"})
schedule.head(2)

,schedule_items,deleted,name,id_schedule,user_id,created_at
0,"[{'name': 'DAA A', 'day': 'Senin', 'room': 'A2...",False,212/plan-1,6104d2cf8aadd383ec57d3c3,60f4f2b38cd53dc0e3b371ba,2021-07-31T04:34:23.663Z
1,"[{'name': 'Aljabar Linier A', 'day': 'Rabu', '...",False,Bismillah Sem 3,6104d66a8aadd383ec57d3c4,60f58061752be98a2e59f66a,2021-07-31T04:49:46.213Z


In [ ]:
user_schedule = schedule[["id_schedule", "user_id", "name", "created_at", "deleted"]]
user_schedule.to_csv(HOME+'user_schedule.csv', index=False)

In [ ]:
courses = pd.read_csv(HOME+"courses.csv")
courses.head(2)

,name,credit,term,classes,period_id,id
0,Analisis Numerik,3,5,"[{'name': 'Analisis Numerik A', 'schedule_item...",60f4fab98cd53dc0e3b371c9,0
1,Basis Data,4,4,"[{'name': 'Basis Data A', 'schedule_items': [{...",60f4fab98cd53dc0e3b371c9,1


In [ ]:
# Drop course with duplicate name,credit,term,classes
# Make a dictionary (mapping course_name to course_id)
slice_course = courses[['name','credit','term','classes']]
course_no_dup = slice_course.drop_duplicates(keep="first")
clean_courses = courses.loc[course_no_dup.index].sort_values(by='name')
course_dict = dict(zip(clean_courses['name'], clean_courses['id']))

In [ ]:
new_rows = []
course_schedule = pd.DataFrame()

# Get schedule id, cours name, class name, course id
def get_schedule(row):
  for schedule in row['schedule_items']:
    new_schedule = [row['id_schedule'], schedule['course_name'], schedule['name'], course_dict.get(schedule['course_name'])]
    new_rows.append(new_schedule)

schedule.iloc[:,[0,3]].progress_apply(get_schedule, axis=1)
course_schedule = course_schedule.append(new_rows).rename(columns={0: 'id_schedule', 1:'course_name', 2:'class_name', 3:'course_id'})
course_schedule['course_id'].fillna(-1, inplace=True)
course_schedule.to_csv(HOME+'course_schedule.csv', index=False)
course_schedule

  0%|          | 0/5900 [00:00<?, ?it/s]

,id_schedule,course_name,class_name,course_id
0,6104d2cf8aadd383ec57d3c3,Desain & Analisis Algoritma,DAA A,10779.0
1,6104d2cf8aadd383ec57d3c3,Desain & Analisis Algoritma,DAA A,10779.0
2,6104d2cf8aadd383ec57d3c3,Kecerdasan Artifisial & Sains Data Dasar,KASDD D,10.0
3,6104d2cf8aadd383ec57d3c3,Kecerdasan Artifisial & Sains Data Dasar,KASDD D,10.0
4,6104d2cf8aadd383ec57d3c3,Pemrograman Fungsional,Pemrog Fungsional,15.0
...,...,...,...,...
50723,61212abca4fd317044ab8c44,Pemrograman Berbasis Platform,PBP E,38.0
50724,61212abca4fd317044ab8c44,Sistem Operasi untuk Sistem Informasi,SOSI B,39.0
50725,61212abca4fd317044ab8c44,Struktur Data & Algoritma,SDA A,30.0
50726,61212abca4fd317044ab8c44,Struktur Data & Algoritma,SDA A,30.0
